# 🧩 Scenario Notebook: CINEASTE Template

This notebook builds and solves an electricity system planning scenario:
1. Import libraries & configure reproducibility
2. Define techno-economic hypotheses
3. Construct representative weeks & demand profile
4. Instantiate technologies (nuclear, gas, renewables, hydro, storage)
5. Build and solve an optimization model (Docplex)
6. Inspect solution & next steps

Use the markdown and inline comments as a guide to adapt parameters.


In [39]:
# ==== 1. Imports & Reproducibility ====
name_simulation = 'nuke_template'  # Scenario tag used for output folders

# Reproducibility: single RNG seed (NumPy Generator)
RNG_SEED = 42
import os, copy
import numpy as np
import pandas as pd
from collections import Counter
rng = np.random.default_rng(RNG_SEED)

# Local source path for project modules (raw_import aggregates common imports)
path_src = '../../src/'
%run -i {path_src}raw_import.py

# Output directory structure
current_path = os.getcwd()
os.makedirs(f"{current_path}/out/{name_simulation}/input", exist_ok=True)
os.makedirs(f"{current_path}/out/{name_simulation}/output", exist_ok=True)

print(f"Simulation: {name_simulation} | Seed={RNG_SEED}")

Simulation: nuke_template | Seed=42


# Plots

In [ ]:
# ==== 2. Plot configuration ====
fig_w, fig_h = 750, 350

# Toggle source / output diagnostic plots
plot_input  = True
plot_output = True
print_m4    = True  # Whether to print info about representative weeks clustering

# Which input plots to display (if plot_input is True)
Display_input = {
    'demand': True,
    'nuclear_hist': True,
    'fix_costs': True,
    'tot_costs': True,
    'hist_data': True,
    'carbon_cost': True,
    'week_demand': True,
    'week_pv': True,
    'week_won': True,
    'week_wof': True,
    'load_factor_total': False,
    'load_factor_data': False
}

print("Plot configuration loaded")

# Hypothesis

In [ ]:
# ==== 3. Global hypotheses ====
# Representative weeks selection method and scenario-wide economic parameters.
number_of_mean_weeks = 4               # Number of representative weeks retained
profil_weeks = 'M4'                    # {'average','maxmin','M4'} selection strategy
PROFIL_CHOICES = {'average','maxmin','M4'}
assert profil_weeks in PROFIL_CHOICES, f'profil_weeks must be in {PROFIL_CHOICES}'
assert 52 % number_of_mean_weeks == 0, 'number_of_mean_weeks must divide 52'

# Economic high-level parameters
r = 0.04                # Discount rate
cost_co2_2020 = 50       # €/tCO2 base year
cost_co2_2050 = 1000     # €/tCO2 target year for extrapolation

# Demand assumptions
demand_growth = 0.01     # Annual demand growth rate (compound)

# Operational modeling flags
ramping = True           # If True, ramping constraints could be added later

print("Hypotheses defined")

# Nuclear hypothesis

In [ ]:
# ==== 4. Nuclear parameters ====
# Lifetime & investment ceiling (aggregated capacity cap)
nuclear_hist_lifetime = 50            # Lifetime for historic fleet sensitivity (40/50/60)
nuclear_invest_max = 100 * 1.650e3    # Upper bound new nuclear capacity additions (MW)

# New nuclear build timing & flexibility
nuke_new_start = 2035
nuke_new_rup = 0.10                   # Ramp up share of nominal per hour
nuke_new_rdo = nuke_new_rup           # Ramp down symmetric

print("Nuclear hypothesis registered")

# Gas

In [ ]:
# ==== 5. Gas parameters ====
ccgt_bioch4_new_start = 2035  # Year after which methane substitution considered
ccgt_invest_max       = 100e3 # Max yearly investment envelope (MW)
ccgt_bioch4_invest_max= 100e3 # Max yearly investment for bioCH4 variant (MW)
print("Gas hypothesis registered")

# Renewables

In [ ]:
# ==== 6. Variable renewables (VRE) parameters ====
# Load factor scenario choices
LOAD_FACTOR_CHOICES = {'low','medium','high','random'}
# Occ = overnight construction cost scenario choices
OCC_CHOICES = {'low','medium','high','random'}

# Nominal choices for each VRE family
loadfactor_won = 'medium'
loadfactor_wof = 'medium'
loadfactor_pv  = 'medium'
for tag, val in {'won': loadfactor_won, 'wof': loadfactor_wof, 'pv': loadfactor_pv}.items():
    assert val in LOAD_FACTOR_CHOICES, f'loadfactor_{tag} must be in {LOAD_FACTOR_CHOICES}'

# Investment caps (MW)
pv_invest_max  = 100e3
won_invest_max = 100e3
wof_invest_max = 100e3

# Run-of-river hydro treated similarly via load factor scenario
loadfactor_ror = 'medium'
assert loadfactor_ror in LOAD_FACTOR_CHOICES, 'loadfactor_ror invalid'

# OCC scenario choices
occ_won = 'medium'
occ_wof = 'medium'
occ_pv  = 'medium'
for tag, val in {'won': occ_won, 'wof': occ_wof, 'pv': occ_pv}.items():
    assert val in OCC_CHOICES, f'occ_{tag} must be in {OCC_CHOICES}'

print("VRE parameters registered")

# Batteries

In [ ]:
# ==== 7. Storage parameters ====
occ_bat = 'medium'
assert occ_bat in OCC_CHOICES, 'occ_bat invalid'
bat_invest_max = 100e3  # Max battery deployment (MW capacity basis)
print("Storage parameters registered")

# Model - Time

In [40]:
# ==== 8. Time sets ====
# Define master temporal indices: world years (for interpolation) vs scenario years vs weeks vs hours.
start_world       = 1950
start_of_scenario = 2023
end_of_scenario   = 2060
years       = range(start_of_scenario, end_of_scenario + 1)  # Scenario years
weeks       = range(1, number_of_mean_weeks + 1)             # Representative weeks
hours       = range(1, 24*7 + 1)                             # Hour index inside each representative week
years_world = range(start_world, end_of_scenario + 1)        # Full range for interpolation routines
U = np.arange(0, 8760, 1)                                    # Hour vector for profile plotting
print("Time sets defined")

Time sets defined


In [ ]:
# ==== 9. Legacy helper: create_val_dictionary ====
# Some legacy parameter classes expect a helper that interpolates values across years_world.
import numpy as np

def create_val_dictionary(vals, yrs):
    """Interpolate or broadcast a list of values across years_world.
    - vals: list/sequence of numeric values
    - yrs: corresponding list/sequence of years
    Returns dict(year->value).
    """
    if (not vals) or (not yrs):
        return {y: 0.0 for y in years_world}
    if len(vals) != len(yrs):
        raise ValueError('vals and yrs length mismatch')
    if len(vals) == 1:
        return {y: float(vals[0]) for y in years_world}
    interpolated = np.interp(list(years_world), yrs, vals)
    return {y: float(v) for y, v in zip(years_world, interpolated)}
print('Legacy create_val_dictionary wrapper defined')

Legacy create_val_dictionary wrapper defined


In [41]:
# ==== 10. Parameter & technology class imports ====
# Load class definitions for economic, technical and specific (dispatchable, fatal, storage) parameters
%run -i ../../src/classes/parameters/generic/class_prm_eco.py
%run -i ../../src/classes/parameters/generic/class_prm_tech.py
%run -i ../../src/classes/parameters/specific/class_prm_dispatchable.py
%run -i ../../src/classes/parameters/specific/class_prm_fatal.py
%run -i ../../src/classes/parameters/specific/class_prm_storage.py
%run -i ../../src/classes/class_technos.py
print('Parameter & technology classes loaded')

Parameter & technology classes loaded


# ==== 11. Technology instantiation ====
Below we run individual scripts that build each technology's economic and technical parameter objects
and push them into the `techno` dictionary. Each script relies on the time sets and helper classes loaded earlier.
The order matters for reproducibility but not for correctness (no cross-dependency between technologies).

In [43]:
# Instantiate all technologies via legacy script runners
techno = {}
index = 1
%run -i nuclear/historic.py
%run -i nuclear/new.py
%run -i gas/ccgt.py
%run -i gas/ccgt_bioch4.py
%run -i ren/won.py '../../../data/formatted/ren/wind/onshore/2019.inc'
%run -i ren/wof.py '../../../data/formatted/ren/wind/offshore/2019.inc'
%run -i ren/pv.py '../../../data/formatted/ren/solar/pv/2019.inc'
%run -i hydro/ror.py '../../../data/formatted/hydro/ror/2019.inc'
%run -i hydro/lake.py '../../../data/formatted/hydro/lake/2019.inc'
%run -i gas/cogen.py '../../../data/formatted/gas/cogen/2019.inc'
%run -i storage/step.py
%run -i storage/battery.py
print('-'*50)
print('Importing technos ... OK')
print(f"Loaded {len(techno)} technologies")
print('-'*50)

--------------------------------------------------
Importing technos ... OK
Loaded 14 technologies
--------------------------------------------------


# ==== 12. Demand modeling ====
Demand is aggregated to representative weeks using one of three strategies:
- average: simple equal-sized grouping
- maxmin: pick extreme (max/min) + random samples
- M4: hierarchical adjacent-week clustering (Ward distance)
Supporting functions are loaded below.

In [ ]:
# Helper distance metrics for clustering
%run -i ../../src/func/distance_cluster.py
print('distance_cluster functions loaded')

distance_cluster functions loaded


In [ ]:
# Hierarchical clustering routine (R_M4_Demand)
%run -i ../../src/func/R_M4_Demand.py
print('R_M4_Demand loaded')

R_M4_Demand loaded


In [42]:
# Build representative demand dictionary over scenario years
# 1. Read raw hourly demand for base year
# 2. Aggregate into representative weeks according to profil_weeks strategy
# 3. Scale future years with annual growth rate
# 4. Create weight dictionary (# of original weeks mapping into each representative week)

demand = np.loadtxt('../../../data/formatted/demand/2019.inc')[:int(7*24*52)]
demand_reshape = pd.DataFrame(np.array(demand).reshape(-1, 7*24))
demand_total = demand_reshape.values.sum()

if profil_weeks == 'average':
    group_size = 52 // number_of_mean_weeks
    group = np.arange(len(demand_reshape)) // group_size
    demand_average = demand_reshape.groupby(group).mean()
elif profil_weeks == 'maxmin':
    max_row_index = demand_reshape.idxmax().iloc[0]
    max_row = demand_reshape.loc[[max_row_index]]
    min_row_index = demand_reshape.idxmin().iloc[0]
    min_row = demand_reshape.loc[[min_row_index]]
    random_rows_indices = rng.choice(demand_reshape.index, size=number_of_mean_weeks-2, replace=False)
    random_rows = demand_reshape.loc[random_rows_indices]
    demand_average = pd.concat([max_row, min_row, random_rows])
    demand_average.reset_index(drop=True, inplace=True)
    demand_average_total = demand_average.values.sum()
    # Rescale so total annual remains coherent
    demand_average = demand_average * demand_total / (demand_average_total * 52/number_of_mean_weeks)
elif profil_weeks == 'M4':
    group = R_M4_Demand(number_of_mean_weeks, demand_reshape, print_info=print_m4)
    demand_average = demand_reshape.groupby(group).mean()
else:
    raise ValueError(f'Invalid profil_weeks={profil_weeks}')

# Build hourly demand dictionary with growth
demand_dict = {
    (y, w, h): demand_average.iloc[w-1, h-1] * (1 + demand_growth) ** (y - years[0])
    for y in years for w in weeks for h in hours
}

# Representative week weights (how many original weeks map to each)
weight_week_dict = dict(Counter(group))
weight_week_dict = {k+1: v for k, v in weight_week_dict.items()}

print('-'*50)
print('Importing demand ... OK')
print(f"Representative weeks: {number_of_mean_weeks} | Strategy: {profil_weeks}")
print('-'*50)

--------------------------------------------------
Importing demand ... OK
Representative weeks: 4 | Strategy: M4
--------------------------------------------------


# ==== 13. (Deprecated duplicate production cell) ====
This unused production cell will be removed in a future cleanup. Keep the executed production cell above (#11).

# ==== 14. Optimization model setup ====
We instantiate a Docplex model for linear cost minimization. Variables are defined over:
- Capacity (var_P) and investment/decommission flows (var_Inv, var_Dec)
- Hourly production (var_E), curtailment (var_EC), storage energy (var_SE)
Additional constraints (ramping, storage balance) can be added in future iterations.

In [44]:
# Initialize optimization model and progress listener helper
%run -i ../../src/func/progress_bar.py
opt_model = cpx.Model(name=name_simulation)
print('Optimization model initialized')

Optimization model initialized


In [45]:
# ==== 15. Pre-variable preparation ====
# Clear model if re-running & initialize helper dictionaries for cost tracking
opt_model.clear()
max_P = max(demand_dict.values())          # Upper bound for production-related variables
annuities = {}                             # Annualized CAPEX/Depreciation expressions
annual_fix_cost_tot = {}                   # Aggregated fixed cost (annuity + OM + misc + decommission)
print('Model cleared and pre-variable structures initialized')

Model cleared and pre-variable structures initialized


In [46]:
# ==== 16. Decision variables ====
from itertools import product
# Capacity & flow variables
var_P = {(i,y): opt_model.continuous_var(name=f'var_P_{i}_{y}', lb=0,     ub=max_P)      for i,y in product(techno.keys(), years)}
var_Inv = {(i,y): opt_model.continuous_var(name=f'var_Inv_{i}_{y}', lb=0,  ub=max_P)     for i,y in product(techno.keys(), range(years.start-1, years.stop-1))}
var_Dec = {(i,y): opt_model.continuous_var(name=f'var_Dec_{i}_{y}', lb=0,  ub=max_P)     for i,y in product(techno.keys(), range(years.start-1, years.stop-1))}
# Hourly operational variables
var_E = {(i,y,w,h): opt_model.continuous_var(name=f'var_E_{i}_{y}_{w}_{h}',   lb=-max_P, ub=max_P) for i,y,w,h in product(techno.keys(), years, weeks, hours)}
var_EC= {(i,y,w,h): opt_model.continuous_var(name=f'var_EC_{i}_{y}_{w}_{h}',  lb=0)               for i,y,w,h in product(techno.keys(), years, weeks, hours)}
var_SE= {(i,y,w,h): opt_model.continuous_var(name=f'var_SE_{i}_{y}_{w}_{h}',  lb=0,     ub=max_P*8760) for i,y,w,h in product(techno.keys(), years, weeks, hours)}
print('-'*50)
print('Variable declaration ... OK')
print(f"Variables: P={len(var_P)}, Inv={len(var_Inv)}, Dec={len(var_Dec)}, E={len(var_E)}")
print('-'*50)

--------------------------------------------------
Variable declaration ... OK
Variables: P=532, Inv=532, Dec=532, E=357504
--------------------------------------------------


In [47]:
# ==== 17. Demand satisfaction constraints ====
# For each hour of each representative week, aggregate supply equals demand.
sum_var_E = {(y,w,h): opt_model.sum(var_E[i,y,w,h] for i in techno.keys()) for y in years for w in weeks for h in hours}
for y in years:
    for w in weeks:
        for h in hours:
            opt_model.add_constraint(ct=sum_var_E[y,w,h] == demand_dict[y,w,h])
print('Demand balance constraints added')

Demand balance constraints added


In [48]:
# ==== 18. Technology-specific placeholder constraints ====
# Placeholder loop for future constraints (ramping, min up/down, storage balance, availability, etc.)
from tqdm import tqdm
for i, t in tqdm(techno.items(), desc='Technologies'):
    # Example future hook:
    # tec = t.get_tech(); eco = t.get_eco(); spec = t.get_spec()
    # Add ramping constraints if ramping flag True
    pass
print('Technology loop placeholder executed')

Technologies: 100%|██████████| 14/14 [00:00<00:00, 130780.08it/s]

Technology loop placeholder executed


# ==== 19. Cost function & solve ====
We build the discounted objective:
- Fixed cost component (annuities or depreciation + OM + misc + decommissioning)
- Variable cost component (fuel + OM + CO2 + misc) weighted by representative week occurrence
- Residual value term subtracting remaining value of investments at horizon end
Finally, we solve the linear model with CPLEX parameters (threads, MIP gap though model is LP).

In [51]:
# Compute cost components & define objective
for i, t in techno.items():
    tec = t.get_tech(); eco = t.get_eco()
    for y in years:
        if eco.is_cap():  # Annuitized CAPEX path
            annuities[i,y] = opt_model.sum(
                (tec.get_historic_data('INV')[yp-1]*eco.get_fix_cap()[yp-1] if yp < start_of_scenario else var_Inv[i,yp-1]*eco.get_fix_cap()[yp-1])
                for yp in range(y-eco.get_lt(), y+1)
                if (y-eco.get_lt()) <= yp <= y and yp in eco.get_fix_cap()
            )
        else:              # Depreciation path
            annuities[i,y] = eco.get_fix_dep()[y] * var_P[i,y]
        annual_fix_cost_tot[i,y] = annuities[i,y] + (eco.get_fix_om()[y] + eco.get_fix_mi()[y]) * var_P[i,y] + eco.get_deco_cost() * var_Dec[i,y-1]

# Discount factors
lev = {y: 1 / (1 + r) ** (y - start_of_scenario) for y in years}

# Residual value approximation (remaining lifetime fraction of final investments)
res = {}
for i, t in techno.items():
    tec = t.get_tech(); eco = t.get_eco()
    if tec.get_isPvar():
        res[i] = 0
        for y in years[:-1]:
            if tec.get_isPvar()[y]:
                age = years.stop - 1 - y
                remaining = max(0, eco.get_lt() - age)
                res[i] += var_Inv[i,y] * remaining / eco.get_lt()

# Aggregations
sum_fixed_cost = {y: opt_model.sum(annual_fix_cost_tot[i,y] for i in techno.keys()) for y in years}
sum_variable_cost = {
    (y,w,h): opt_model.sum(var_E[i,y,w,h] * techno[i].get_eco().get_var_tot()[y] * weight_week_dict[w] for i in techno.keys())
    for y in years for w in weeks for h in hours
}
objective = opt_model.sum(sum_fixed_cost[y] * lev[y] for y in years) \
          + opt_model.sum(sum_variable_cost[y,w,h] * lev[y] for y in years for w in weeks for h in hours) \
          - opt_model.sum(res[i] * lev[years.stop-1] for i in techno.keys())

# Solver configuration
cplex_params = opt_model.context.cplex_parameters
cplex_params.threads = 8
cplex_params.mip.tolerances.mipgap.set(0.01)  # Harmless for LP
opt_model.minimize(objective)
progress_listener = MyProgressListener(interval=1)
opt_model.add_progress_listener(progress_listener)
solution = opt_model.solve(log_output=True)
print('--- Solution status ---')
print('Status:', opt_model.solve_details.status)
if solution is None:
    raise RuntimeError('No solution obtained')
print('Objective:', format(opt_model.objective_value, ' .2e'), '€')
print('Solve time:', format(opt_model.solve_details.time, ' .3g'), 's')

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 8
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 8
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...

Iteration log . . .
Iteration:     1   Dual objective     = -18430242352895.132812

Iteration log . . .
Iteration:     1   Dual objective     = -18430242352895.132812
Perturbation s

# ==== 20. Notes & Next Steps ====
This refactored template improves readability and structure:
- Clear section headers & inline comments
- Normalized prints and variable naming
- Removed duplicate production cell
- Added residual value term and explanatory markdown

Potential enhancements (future work):
1. Add ramping / availability / storage balance constraints
2. Replace legacy %run calls with module imports for better dependency tracking
3. Implement validation plots (dispatch vs demand, capacity trajectories)
4. Integrate emissions accounting & carbon budget constraint
5. Add unit tests for helper functions (dictionary interpolation, clustering)

To adapt scenario: adjust hypotheses (discount rate, demand growth, technology cost caps) in sections 3–7.

> After modifications, re-run Cells 2, 8–12, 15–19 in order.

# ==== 21. Additional Constraints (Investment, Production, Ramping, Storage) ====
# This cell adds detailed constraints similar to the original reference model.
# It should be (re)run after variables are declared and before rebuilding objective if changed.


In [50]:
# Detailed constraints derived from reference model logic
from tqdm import tqdm

# Remove previously added simplistic demand constraints & re-add full structure if re-running
# (If this is a first run after kernel restart, they don't exist yet.)
# NOTE: Docplex doesn't expose direct removal; in iterative dev restart kernel for clean rebuild.

# Precompute aggregate supply expression
sum_var_E = {(y,w,h): opt_model.sum(var_E[i,y,w,h] for i in techno.keys()) for y in years for w in weeks for h in hours}

# Demand equality (curtailment handled by var_EC in fatal formulation)
for y in years:
    for w in weeks:
        for h in hours:
            opt_model.add_constraint(sum_var_E[y,w,h] == demand_dict[y,w,h])

# Technology-level constraints
for i, t in tqdm(techno.items(), desc='Constraints techno'):
    tec  = t.get_tech(); eco = t.get_eco(); spec = t.get_spec()
    is_storage = (t.get_type() == 'storage')
    is_fatal   = (t.get_type() == 'fatal')
    is_P_var   = tec.get_isPvar()
    is_E_var   = tec.get_isEvar()

    # Enforce positivity for non-storage production and disable storage level use
    if not is_storage:
        for y in years:
            for w in weeks:
                for h in hours:
                    opt_model.add_constraint(var_E[i,y,w,h] >= 0)
                    if (i,y,w,h) in var_SE:  # storage energy level variable present only for storage
                        opt_model.add_constraint(var_SE[i,y,w,h] == 0)

    # Deployment timing thresholds (example: new nuclear, bioCH4 CCGT)
    if t.get_title() == 'new':
        for y in range(years.start, nuke_new_start):
            opt_model.add_constraint(var_P[i,y] == 0)
    if t.get_title() == 'ccgt_bioch4':
        for y in range(years.start, ccgt_bioch4_new_start):
            opt_model.add_constraint(var_P[i,y] == 0)

    # Investment / Decommission dynamics
    for y in years:
        if is_P_var[y]:
            # Investment cap
            InvMax = tec.get_InvMax()
            if InvMax is not None:
                opt_model.add_constraint(var_Inv[i,y-1] <= InvMax[y-1])
            # Decommission equals historic investments shifted by lifetime OR endogenous investment
            if (y - eco.get_lt()) <= years.start - 1:
                hist_inv = tec.get_historic_data('INV')
                if hist_inv and (y-1-eco.get_lt()) in hist_inv:
                    opt_model.add_constraint(var_Dec[i,y-1] == hist_inv[y-1 - eco.get_lt()])
                else:
                    opt_model.add_constraint(var_Dec[i,y-1] == 0)
            else:
                opt_model.add_constraint(var_Dec[i,y-1] == var_Inv[i,y-1 - eco.get_lt()])
            # Capacity linking (first year uses historic capacity)
            if y == years.start:
                hist_capa = tec.get_historic_data('CAPA')
                base_capa = hist_capa.get(y-1, 0) if hist_capa else 0
                opt_model.add_constraint(var_P[i,y] == base_capa + var_Inv[i,y-1] - var_Dec[i,y-1])
            else:
                opt_model.add_constraint(var_P[i,y] == var_P[i,y-1] + var_Inv[i,y-1] - var_Dec[i,y-1])
        else:
            # Exogenous capacity path
            opt_model.add_constraint(var_Inv[i,y-1] == 0)
            opt_model.add_constraint(var_Dec[i,y-1] == 0)
            P_exo = tec.get_P()
            if P_exo:
                opt_model.add_constraint(var_P[i,y] == P_exo[y])

    # Production upper bound (availability)
    A = tec.get_A()
    for y in years:
        for w in weeks:
            avail = A[y,w] if A is not None else 1
            for h in hours:
                opt_model.add_constraint(var_E[i,y,w,h] <= var_P[i,y] * avail)
                if not is_E_var[y,w,h]:
                    # Exogenous energy path
                    E_exo = tec.get_E()
                    if E_exo:
                        opt_model.add_constraint(var_E[i,y,w,h] == E_exo[y,w,h])

    # Fatal technologies energy definition with curtailment
    if is_fatal:
        LF = tec.get_LF()
        for y in years:
            for w in weeks:
                for h in hours:
                    lf_val = LF[y,w,h] if LF else 0
                    opt_model.add_constraint(var_E[i,y,w,h] == lf_val * var_P[i,y] - var_EC[i,y,w,h])

    # Ramping for non-fatal dispatchable
    if ramping and not is_fatal and hasattr(spec,'get_rup'):
        rup = spec.get_rup(); rdo = spec.get_rdo()
        for y in years:
            for w in weeks:
                for h in hours:
                    if is_E_var[y,w,h]:
                        # Determine next (y,w,h)
                        if h == hours.stop-1:
                            if w == weeks.stop-1:
                                if y == years.stop-1:
                                    continue
                                # Next year first hour
                                opt_model.add_constraint(var_E[i,y+1,1,1] <= var_E[i,y,w,h] + var_P[i,y+1]*rup)
                                opt_model.add_constraint(var_E[i,y+1,1,1] >= var_E[i,y,w,h] - var_P[i,y+1]*rdo)
                            else:
                                # Next week first hour
                                opt_model.add_constraint(var_E[i,y,w+1,1] <= var_E[i,y,w,h] + var_P[i,y]*rup)
                                opt_model.add_constraint(var_E[i,y,w+1,1] >= var_E[i,y,w,h] - var_P[i,y]*rdo)
                        else:
                            # Next hour in same week
                            opt_model.add_constraint(var_E[i,y,w,h+1] <= var_E[i,y,w,h] + var_P[i,y]*rup)
                            opt_model.add_constraint(var_E[i,y,w,h+1] >= var_E[i,y,w,h] - var_P[i,y]*rdo)

    # Storage charge/discharge balance
    if is_storage and t.get_title() == 'charge':
        i_charge = i
        i_discharge = i + 1  # Assumes paired ordering
        eff_charge    = spec.get_efficiency_charge()
        eff_discharge = spec.get_efficiency_discharge()
        level_max_time = spec.get_level_max_time()
        level_min      = spec.get_level_min()
        level_start    = spec.get_level_start()
        for y in years:
            if spec.get_is_P_sym():
                opt_model.add_constraint(var_P[i_discharge,y] == var_P[i_charge,y])
            for w in weeks:
                for h in hours:
                    # Bounds and sign conventions
                    opt_model.add_constraint(var_E[i_discharge,y,w,h] <= var_P[i_discharge,y])
                    opt_model.add_constraint(var_E[i_discharge,y,w,h] >= 0)
                    opt_model.add_constraint(var_E[i_charge,y,w,h] <= 0)
                    opt_model.add_constraint(-var_E[i_charge,y,w,h] <= var_P[i_charge,y])
                    # Shared level variable
                    opt_model.add_constraint(var_SE[i_charge,y,w,h] == var_SE[i_discharge,y,w,h])
                    # Level recursion
                    if y == years[0] and w == weeks[0] and h == hours[0]:
                        opt_model.add_constraint(var_SE[i_charge,y,w,h] == level_start)
                    elif h == hours[0]:
                        if w == weeks[0]:
                            prev_level = var_SE[i_charge,y-1,weeks[-1],hours[-1]] if y!=years[0] else level_start
                            prev_start = var_SE[i_charge,y-1,weeks[-1],hours[0]] if y!=years[0] else level_start
                            opt_model.add_constraint(var_SE[i_charge,y,w,h] == -eff_charge*var_E[i_charge,y,w,h] - (1/eff_discharge)*var_E[i_discharge,y,w,h] + (prev_level - prev_start)*weight_week_dict[weeks[-1]] + prev_start)
                        else:
                            prev_level = var_SE[i_charge,y,w-1,hours[-1]]
                            prev_start = var_SE[i_charge,y,w-1,hours[0]]
                            opt_model.add_constraint(var_SE[i_charge,y,w,h] == -eff_charge*var_E[i_charge,y,w,h] - (1/eff_discharge)*var_E[i_discharge,y,w,h] + (prev_level - prev_start)*weight_week_dict[w-1] + prev_start)
                    else:
                        opt_model.add_constraint(var_SE[i_charge,y,w,h] == -eff_charge*var_E[i_charge,y,w,h] - (1/eff_discharge)*var_E[i_discharge,y,w,h] + var_SE[i_charge,y,w,h-1])
                    # Level bounds
                    opt_model.add_constraint(var_SE[i_charge,y,w,h] <= level_max_time * var_P[i_charge,y])
                    opt_model.add_constraint(var_SE[i_charge,y,w,h] >= level_min)
print('Extended constraints added')

Constraints techno: 100%|██████████| 14/14 [01:34<00:00,  6.72s/it]

Extended constraints added


# ==== 22. Store Solution Variables ====
# After solving, extract variable values back into techno objects and prepare output plot flags.


In [52]:
# Persist solution values into techno objects (capacity, investment, decommission, energy, storage levels)
if 'solution' not in globals() or solution is None:
    print('No solution available yet. Run objective/solve cell before storing results.')
else:
    for i, t in tqdm(techno.items(), desc='Store techno'):
        is_storage = (t.get_type() == 'storage')
        P_hist = {}; P_new = {}; Inv_hist = {}; Inv_new = {}; Dec_hist = {}; Dec_new = {}; E_sol = {}
        tec = t.get_tech(); eco = t.get_eco()
        hist_capa = tec.get_historic_data('CAPA') or {}
        hist_inv  = tec.get_historic_data('INV') or {}
        hist_dec  = tec.get_historic_data('DEC') or {}
        for y in years_world[1:]:
            if y < start_of_scenario:
                if y in hist_capa: P_hist[y] = hist_capa[y]
                if y-1 in hist_dec: Dec_hist[y-1] = hist_dec[y-1]
                if y-1 in hist_inv: Inv_hist[y-1] = hist_inv[y-1]
            else:
                P_new[y] = var_P[i,y].solution_value
                Dec_new[y-1] = var_Dec[i,y-1].solution_value
                Inv_new[y-1] = var_Inv[i,y-1].solution_value
                for w in weeks:
                    for h in hours:
                        E_sol[y,w,h] = var_E[i,y,w,h].solution_value
        if is_storage:
            SE = {(y,w,h): var_SE[i,y,w,h].solution_value for y in years for w in weeks for h in hours}
            t.get_spec().set_level(copy.deepcopy(SE))
        tec.set_P(copy.deepcopy({**P_hist, **P_new}))
        tec.set_Dec(copy.deepcopy({**Dec_hist, **Dec_new}))
        tec.set_Inv(copy.deepcopy({**Inv_hist, **Inv_new}))
        tec.set_E(copy.deepcopy(E_sol))
    print('Solution values stored.')

# Output display flags (reuse or adjust)
year_start = years.stop-1
week_start = 1
nombre_week_affichage = number_of_mean_weeks
Display_output = {
    'production': True,
    'stock': True,
    'capacity': True,
    'mix': True,
    'mix_frac': True,
    'inv_dec_capa': True,
}
print('Display_output flags configured')

Store techno: 100%|██████████| 14/14 [00:02<00:00,  5.91it/s]

Solution values stored.
Display_output flags configured


# ==== 23. Plot Results ====
# Use legacy plotting module to generate HTML plots for production, stock, capacity, mix, and inv/dec/capa.


In [53]:
# Plot output figures
%run -i ../../src/func/plot_output.py
print('Result plots generated and saved to out/', name_simulation)

Result plots generated and saved to out/ nuke_template
